# Bootcamp: Cientista de Dados - Desafio Final

- Data: julho de 2022.

## Autor

Feito por [Alexsander Lopes Camargos](https://github.com/alexcamargos): Entre em contato!

[![GitHub](https://img.shields.io/badge/-AlexCamargos-1ca0f1?style=flat-square&labelColor=1ca0f1&logo=github&logoColor=white&link=https://github.com/alexcamargos)](https://github.com/alexcamargos)
[![Twitter Badge](https://img.shields.io/badge/-@alcamargos-1ca0f1?style=flat-square&labelColor=1ca0f1&logo=twitter&logoColor=white&link=https://twitter.com/alcamargos)](https://twitter.com/alcamargos)
[![Linkedin Badge](https://img.shields.io/badge/-alexcamargos-1ca0f1?style=flat-square&logo=Linkedin&logoColor=white&link=https://www.linkedin.com/in/alexcamargos/)](https://www.linkedin.com/in/alexcamargos/)
[![Gmail Badge](https://img.shields.io/badge/-alcamargos@vivaldi.net-1ca0f1?style=flat-square&labelColor=1ca0f1&logo=Gmail&logoColor=white&link=mailto:alcamargos@vivaldi.net)](mailto:alcamargos@vivaldi.net)

## Licença

[MIT License](https://choosealicense.com/licenses/mit/)

## Objetivos

- Exercitar os seguintes conceitos trabalhados no Bootcamp:
- Pré-processamento dos dados.
- Detecção de anomalias.
- Processamento dos dados.
- Correlações.
- Spark MlLib.
- Interpretação dos dados.

## Enunciado

O derrame é uma das doenças que mais acometem a população mundial. Segundo a World Health Organization (WHO), o Acidente Vascular Cerebral (AVC) foi a segunda maior causa de morte na população mundial no ano de 2016 [https://www.who.int/news-room/fact-sheets/detail/the-top-10-causes-of-death](https://www.who.int/news-room/fact-sheets/detail/the-top-10-causes-of-death).

Neste desafio, vamos realizar uma análise sobre um banco de dados composto por uma pesquisa realizada com diferentes pacientes. Nesta análise, vamos tentar prever, nos baseando em algumas características específicas, se um determinado indivíduo irá ou não sofrer um AVC. Para isso, vamos aplicar o pré-processamento dos dados e a aplicar o modelo de Regressão Logística e o SVM, para indicar se um indivíduo possui ou não maior probabilidade de desenvolver o AVC.

## Atividades

Os alunos deverão desempenhar as seguintes atividades:

1. Acessar o site [https://community.cloud.databricks.com/](https://community.cloud.databricks.com/) e utilizar a conta gratuita. O tutorial de criação da conta gratuita está presente na plataforma Canvas, logo abaixo da apostila, no item “Arquivos complementares” com o nome “tutorial_databricks_TPD.pdf”.
2. Acessar e baixar os arquivos “healthcare_dataset_stroke_data.csv” e o “desafio_CID.ipynb” presentes na pasta: [https://drive.google.com/drive/folders/1alRfzr1WbOVlfvffFo8zrXSlwniW1Nd8?usp=sharing](https://drive.google.com/drive/folders/1alRfzr1WbOVlfvffFo8zrXSlwniW1Nd8?usp=sharing).
3. Responda às perguntas do desafio.

In [ ]:
# Importando os módulos necessários.
from pyspark import SparkFiles
from pyspark.sql import SparkSession

from pyspark.sql import functions as sql_F
from pyspark.sql import types as sql_T

In [ ]:
# Criando o SparkSession
spark  = SparkSession.builder.appName('Desafio Final').getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [ ]:
# Link para acesso ao dataset no GitHub.
DATASET_LINK = 'https://raw.githubusercontent.com/alexcamargos/DataScienceBootcampIGTI/main/DesafioFinal/healthcare-dataset-stroke-data.csv'

In [ ]:
# Informando ao Spark onde esta o arquivo para download.
spark.sparkContext.addFile(DATASET_LINK)

In [ ]:
# Carregando o dataset. Nosso arquivo CSV tem cabeçalho definido e usa encoding Windows-1252.
# header=True - Indica que a primeira linha do arquivo é o cabeçalho.
# inferSchema=True - Tenta determinar o schema a partir dos dados.
df = spark.read.option('header', True) \
               .option('inferSchema', 'True') \
               .csv(f'file://{SparkFiles.get("healthcare-dataset-stroke-data.csv")}')

In [ ]:
# Visualizando o dataframe.
df.show(10)

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
| 9046|  Male|67.0|           0|            1|         Yes|      Private|         Urban|           228.69|36.6|formerly smoked|     1|
|51676|Female|61.0|           0|            0|         Yes|Self-employed|         Rural|           202.21| N/A|   never smoked|     1|
|31112|  Male|80.0|           0|            1|         Yes|      Private|         Rural|           105.92|32.5|   never smoked|     1|
|60182|Female|49.0|           0|            0|         Yes|      Private|         Urban|           171.23|34.4|         smokes|     1|
| 1665|Female|79.0|           1|            0|         

In [ ]:
df.show(1, vertical=True)

-RECORD 0----------------------------
 id                | 9046            
 gender            | Male            
 age               | 67.0            
 hypertension      | 0               
 heart_disease     | 1               
 ever_married      | Yes             
 work_type         | Private         
 Residence_type    | Urban           
 avg_glucose_level | 228.69          
 bmi               | 36.6            
 smoking_status    | formerly smoked 
 stroke            | 1               
only showing top 1 row



In [ ]:
# Quais são as colunas do dataframe.
df.columns

Out[42]: ['id',
 'gender',
 'age',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'avg_glucose_level',
 'bmi',
 'smoking_status',
 'stroke']

In [ ]:
# Visualizando o esquema inferido de forma automática para as colunas.
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: string (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [ ]:
# Verificando a existência ou não de valores nulos no dataframe.
df.select([sql_F.count(sql_F.when(sql_F.isnan(column), column)).alias(column) for column in df.columns]).show(vertical=True)

-RECORD 0----------------
 id                | 0   
 gender            | 0   
 age               | 0   
 hypertension      | 0   
 heart_disease     | 0   
 ever_married      | 0   
 work_type         | 0   
 Residence_type    | 0   
 avg_glucose_level | 0   
 bmi               | 0   
 smoking_status    | 0   
 stroke            | 0   



In [ ]:
# Estatísticas descritiva do dataframe.
display(df.describe())

summary,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
count,5110,5110,5110,5110,5110,5110,5110,5110,5110,5110,5110,5110
mean,36517.82935420744,null,43.226614481409015,0.0974559686888454,0.05401174168297456,null,null,null,106.14767710371804,28.893236911794673,null,0.0487279843444227
stddev,21161.72162482715,null,22.61264672311348,0.296606674233791,0.22606298750336554,null,null,null,45.28356015058193,7.85406672968016,null,0.21531985698023753
min,67,Female,0.08,0,0,No,Govt_job,Rural,55.12,10.3,Unknown,0
max,72940,Other,82.0,1,1,Yes,children,Urban,271.74,N/A,smokes,1


## Pergunta 1 - Quantas instâncias e características existem, respectivamente, no dataset?

In [ ]:
print(f'Nosso dataframe tem {df.count()} linhas (instâncias).')
print(f'Nosso dataframe tem {len(df.columns)} colunas (características).')

Nosso dataframe tem 5110 linhas (instâncias).
Nosso dataframe tem 12 colunas (características).


## Pergunta 2 - Quantas variáveis do tipo “string” estão presentes no dataset?

In [ ]:
len([field.name for field in df.schema.fields if isinstance(field.dataType, sql_T.StringType)])

Out[47]: 6

In [ ]:
num_variables_string = len([column for column, dtype in df.dtypes if dtype == 'string'])
print(f'No dataset temos {num_variables_string} variáveis do tipo string')

No dataset temos 6 variáveis do tipo string


## Pergunta 3 - Qual é a idade (age) média dos entrevistados?

In [ ]:
df.agg({'age': 'mean'}).show(vertical=True)

-RECORD 0----------------------
 avg(age) | 43.226614481409015 



In [ ]:
df.select('age').summary().show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|              5110|
|   mean|43.226614481409015|
| stddev| 22.61264672311348|
|    min|              0.08|
|    25%|              25.0|
|    50%|              45.0|
|    75%|              61.0|
|    max|              82.0|
+-------+------------------+



## Pergunta 4 - Sobre a distribuição de AVC em relação ao sexo (gender) dos entrevistados, é CORRETO afirmar:

In [ ]:
df.groupby(['gender']).agg({'stroke': 'count'}).sort('count(stroke)', ascending=False).show()

+------+-------------+
|gender|count(stroke)|
+------+-------------+
|Female|         2994|
|  Male|         2115|
| Other|            1|
+------+-------------+



## Pergunta 5 - É correto afirmar sobre o dataset, EXCETO:

### Existem dados categóricos e numéricos presentes neste dataset. Um exemplo de dados categóricos é o “Residence_type”.

In [ ]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: string (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



### Existem dois tipos diferentes de classes de residências (“Residence_type”) presentes nesse dataset.

In [ ]:
df.groupby('Residence_type').agg({'Residence_type':'count'}).sort('count(Residence_type)', ascending=False).show()

+--------------+---------------------+
|Residence_type|count(Residence_type)|
+--------------+---------------------+
|         Urban|                 2596|
|         Rural|                 2514|
+--------------+---------------------+



### A variável bmi possui valores não numéricos.

In [ ]:
df.select('bmi').dtypes

Out[54]: [('bmi', 'string')]

In [ ]:
df.select('bmi').show(10)

+----+
| bmi|
+----+
|36.6|
| N/A|
|32.5|
|34.4|
|  24|
|  29|
|27.4|
|22.8|
| N/A|
|24.2|
+----+
only showing top 10 rows



### O dataset está balanceado. Existem quantidades similares de instâncias de indivíduos que sofreram AVC e que não sofreram dessa enfermidade.

In [ ]:
df.groupBy('stroke').agg({'stroke':'count'}).show()

+------+-------------+
|stroke|count(stroke)|
+------+-------------+
|     1|          249|
|     0|         4861|
+------+-------------+



## Pergunta 6 - Qual é o valor da mediana para a variável do nível médio de glicose do entrevistado (“avg_glucose_level”)?

In [ ]:
display(df.select('avg_glucose_level'))

avg_glucose_level
228.69
202.21
105.92
171.23
174.12
186.21
70.09
94.39
76.15
58.57


In [ ]:
df.select('avg_glucose_level').summary().show()

+-------+------------------+
|summary| avg_glucose_level|
+-------+------------------+
|  count|              5110|
|   mean|106.14767710371804|
| stddev| 45.28356015058193|
|    min|             55.12|
|    25%|             77.23|
|    50%|             91.85|
|    75%|            114.09|
|    max|            271.74|
+-------+------------------+



In [ ]:
avg_glucose_level = df.select('avg_glucose_level').toPandas()
avg_glucose_level.median()

Out[59]: avg_glucose_level    91.885
dtype: float64

## Pergunta 7 - Analisando o padrão de dispersão da variável do nível médio de glicose do entrevistado (“avg_glucose_level”), é correto afirmar, EXCETO:

In [ ]:
display(df.select('avg_glucose_level'))

avg_glucose_level
228.69
202.21
105.92
171.23
174.12
186.21
70.09
94.39
76.15
58.57


In [ ]:
df.select('avg_glucose_level').summary().show()

+-------+------------------+
|summary| avg_glucose_level|
+-------+------------------+
|  count|              5110|
|   mean|106.14767710371804|
| stddev| 45.28356015058193|
|    min|             55.12|
|    25%|             77.23|
|    50%|             91.85|
|    75%|            114.09|
|    max|            271.74|
+-------+------------------+



## Pergunta 8 - Analisando a dispersão dos dados para a variável idade (“age”), é correto afirmar, EXCETO:

In [ ]:
display(df.select('age'))

age
67.0
61.0
80.0
49.0
79.0
81.0
74.0
69.0
59.0
78.0


In [ ]:
df.select('age').summary().show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|              5110|
|   mean|43.226614481409015|
| stddev| 22.61264672311348|
|    min|              0.08|
|    25%|              25.0|
|    50%|              45.0|
|    75%|              61.0|
|    max|              82.0|
+-------+------------------+



## Pergunta 9 - Quantas classes diferentes para a variável “work_type” existem no dataset?

In [ ]:
df.groupBy('work_type').agg({'work_type': 'count'}).agg({'count(work_type)': 'count'}).show()

+-----------------------+
|count(count(work_type))|
+-----------------------+
|                      5|
+-----------------------+



## Pergunta 10 - Dentre as classes de tipos de trabalhos existentes (work_type), qual é aquela que possui uma maior quantidade de instâncias?

In [ ]:
df.groupBy('work_type').agg({'work_type': 'count'}).sort('count(work_type)', ascending=False).show()

+-------------+----------------+
|    work_type|count(work_type)|
+-------------+----------------+
|      Private|            2925|
|Self-employed|             819|
|     children|             687|
|     Govt_job|             657|
| Never_worked|              22|
+-------------+----------------+



## Pergunta 12 - Analisando as variáveis “bmi” e “smoking_status”, é CORRETO afirmar:

In [ ]:
df.groupBy('smoking_status').agg({'bmi': 'count'}).sort('count(bmi)', ascending=False).show()

+---------------+----------+
| smoking_status|count(bmi)|
+---------------+----------+
|   never smoked|      1892|
|        Unknown|      1544|
|formerly smoked|       885|
|         smokes|       789|
+---------------+----------+



## Pergunta 13 - Após o agrupamento dos dados de “smoking_status” e ”stroke”, é CORRETO afirmar que:

In [ ]:
df.groupBy('smoking_status').agg({'stroke': 'count'}).sort('count(stroke)', ascending=False).show()

+---------------+-------------+
| smoking_status|count(stroke)|
+---------------+-------------+
|   never smoked|         1892|
|        Unknown|         1544|
|formerly smoked|          885|
|         smokes|          789|
+---------------+-------------+



## Pergunta 14 - Sobre a relação entre a hipertensão (hypertension) e o AVC (stroke) presente neste dataset, é CORRETO afirmar:

In [ ]:
df.groupby('stroke').agg({'hypertension': 'count'}).show()

+------+-------------------+
|stroke|count(hypertension)|
+------+-------------------+
|     1|                249|
|     0|               4861|
+------+-------------------+

